# MusciVAE

- input: 130차원(velocity) -> T, 9
- drum class: 9 canonical classes
- 16개의 note intervals -> 16 events
- 2-bar data -> T=32, 16-bar data -> T=256
- U = 16 -> 1개의 bar 안에 16개

- 2-bar
- input: T, 130, 9
- bar별로 16개의 note들
- rnn model: slide 1bar
- output: U=16

In [ ]:
# dataset structure
"""
    Create a pandas to store the piano rolls in.
    It'll look a bit like this:
    
    |   Index     | t | C0 |...| C10 |
    |Song_name_3:0| 0 | 40 |...| 0   |
    |             | 1 | 40 |...| 0   |
    |             |...|... |...| ... |
    |             | N | 40 |...| 0   |
    |Song_name_3:1| 0 | 40 |...| 0   |
    |             |...|... |...| ... |
    |             | N | 40 |...| 0   |
    |Song_name_4:0| 0 | 40 |...| 0   |
    |             |...|... |...| ... |
    |             | N | 40 |...| 0   |

"""

In [85]:
import pandas as pd
import pretty_midi
import os
from getpass import getuser
import numpy as np
from yullab.utils import class_map
from yullab.prep import generate_midi_df, BarTransform
from yullab.datasets import GrooveDataset

In [13]:
username = getuser()
base_path = f'/Users/{username}/data/midi/groove'
info_csv_filename = os.path.join(base_path, 'info.csv')

info_df = pd.read_csv(info_csv_filename)
info_df

,drummer,session,id,style,bpm,beat_type,time_signature,midi_filename,audio_filename,duration,split
0,drummer1,drummer1/eval_session,drummer1/eval_session/1,funk/groove1,138,beat,4-4,drummer1/eval_session/1_funk-groove1_138_beat_...,drummer1/eval_session/1_funk-groove1_138_beat_...,27.872308,test
1,drummer1,drummer1/eval_session,drummer1/eval_session/10,soul/groove10,102,beat,4-4,drummer1/eval_session/10_soul-groove10_102_bea...,drummer1/eval_session/10_soul-groove10_102_bea...,37.691158,test
2,drummer1,drummer1/eval_session,drummer1/eval_session/2,funk/groove2,105,beat,4-4,drummer1/eval_session/2_funk-groove2_105_beat_...,drummer1/eval_session/2_funk-groove2_105_beat_...,36.351218,test
3,drummer1,drummer1/eval_session,drummer1/eval_session/3,soul/groove3,86,beat,4-4,drummer1/eval_session/3_soul-groove3_86_beat_4...,drummer1/eval_session/3_soul-groove3_86_beat_4...,44.716543,test
4,drummer1,drummer1/eval_session,drummer1/eval_session/4,soul/groove4,80,beat,4-4,drummer1/eval_session/4_soul-groove4_80_beat_4...,drummer1/eval_session/4_soul-groove4_80_beat_4...,47.987500,test
...,...,...,...,...,...,...,...,...,...,...,...
1145,drummer2,drummer2/session2,drummer2/session2/11,rock,130,beat,4-4,drummer2/session2/11_rock_130_beat_4-4.mid,NaN,1.909613,train
1146,drummer2,drummer2/session2,drummer2/session2/12,rock,130,beat,4-4,drummer2/session2/12_rock_130_beat_4-4.mid,NaN,1.808652,train
1147,drummer2,drummer2/session2,drummer2/session2/13,rock,130,beat,4-4,drummer2/session2/13_rock_130_beat_4-4.mid,NaN,1.864421,train
1148,drummer2,drummer2/session2,drummer2/session2/14,rock,130,beat,4-4,drummer2/session2/14_rock_130_beat_4-4.mid,NaN,1.875960,train


In [82]:
sample_filenames = info_df[info_df.split.eq('train')]['midi_filename']
drum_df = generate_midi_df(base_path, sample_filenames)
drum_df.to_csv(os.path.join(base_path, 'midi.csv'))

In [89]:
transform = BarTransform(2)
dataset = GrooveDataset('/Users/rimiiii/data/midi/groove/midi.csv', transform)

In [96]:
dataset[10].shape

(32, 9)